In [1]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

In [2]:
pth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles'

pth_mat = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\10x\matfiles'

WSIs = [_ for _ in os.listdir(pth) if _.endswith('ndpi')]
crop_mats = [_ for _ in os.listdir(pth_mat) if _.endswith('mat')]

In [3]:
WSIs

['monkey_fetus_40_0105_ecm.ndpi',
 'monkey_fetus_40_0271_eye.ndpi',
 'monkey_fetus_40_0400_spinalcord.ndpi',
 'monkey_fetus_40_0420_brain.ndpi',
 'monkey_fetus_40_0432_bone.ndpi',
 'monkey_fetus_40_0450_heart.ndpi',
 'monkey_fetus_40_0730_kidney.ndpi']

In [4]:
crop_mats

['crop_monkey_fetus_40_0105_ecm.mat',
 'crop_monkey_fetus_40_0271_eye.mat',
 'crop_monkey_fetus_40_0400_spinalcord.mat',
 'crop_monkey_fetus_40_0420_brain.mat',
 'crop_monkey_fetus_40_0432_bone.mat',
 'crop_monkey_fetus_40_0450_heart.mat',
 'crop_monkey_fetus_40_0730_kidney.mat']

In [5]:
res20x=0.4416
res10x=1

ratio = res10x/res20x
print(ratio)

2.2644927536231885


In [6]:
from scipy.io import loadmat

res20x=0.4416
res10x=1

ratio = res10x/res20x

outpth = os.path.join(pth,'tiles')
if not os.path.exists(outpth):
    os.mkdir(outpth)

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,WSIs[i][:-5]))}.png'

    if not os.path.exists(f_out_name):
        print(f_out_name)
        print(mat_file_name)
        print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]

        data = loadmat(mat_file_name)

        offset = int(crop_width)

        crop_width = int(data['crop_width']*ratio)
        crop_height = int(data['crop_height']*ratio)
        crop_x = int(data['crop_x']*ratio)
        crop_y = int(data['crop_y']*ratio)

        print(crop_x)
        print(crop_y)
        print(crop_width)
        print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()



In [7]:
# jpeg to png
from PIL import Image

pth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\tiles\jpeg'
jpegs = [_ for _ in os.listdir(pth) if _.endswith('jpeg')]

outpth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\tiles\png'

for i, file in enumerate(jpegs):
    file_name = os.path.join(pth, jpegs[i])
    # Open the TIF image using Pillow
    tif_image = Image.open(file_name)

    # Specify the output file path with a .png extension
    output_path = f'{os.path.join(outpth, jpegs[i][:-5])}.png'

    print(file_name)
    print(output_path)

    # Save the image as PNG
    tif_image.save(output_path, 'PNG')

    # Close the TIF image
    tif_image.close()

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\\\10.99.68.178\\andreex\\data\\monkey fetus\\gestational 40\\1024tiles\\tiles\\jpeg'